In [4]:
print("thilan")

thilan


In [5]:
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader


In [6]:
PINECONE_API_KEY = "pcsk_4kdMmd_LSyLr9FYWARVseBiXahi4q28hE1t3Hpt73vA9mi39yMDP57H18QsPL9Gi7xDzJw"
PINECONE_API_ENV = "quickstart"

In [7]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls=PyPDFLoader)
    document = loader.load()
    return document

In [8]:
extracted_data = load_pdf("data/")

In [9]:
extracted_data

[Document(metadata={'producer': 'Acrobat PDFWriter 4.0 for Windows', 'creator': 'Microsoft Word - WHO_DIAGNOSI_11gW97.doc', 'creationdate': 'D:20000209211438', 'title': 'WHO_DIAGNOSI_11gW97', 'moddate': '2000-02-09T21:58:13+00:00', 'source': 'data\\medical_book.pdf', 'total_pages': 65, 'page': 0, 'page_label': '1'}, page_content='WHO/NCD/NCS/99.2\nOriginal: English\nDistr.: General\nDefinition, Diagnosis\nand Classification\nof Diabetes Mellitus\nand its Complications\nReport of a WHO Consultation\nPart 1: Diagnosis and Classification of\nDiabetes Mellitus\nWorld Health Organization\nDepartment of Noncommunicable Disease Surveillance\nGeneva'),
 Document(metadata={'producer': 'Acrobat PDFWriter 4.0 for Windows', 'creator': 'Microsoft Word - WHO_DIAGNOSI_11gW97.doc', 'creationdate': 'D:20000209211438', 'title': 'WHO_DIAGNOSI_11gW97', 'moddate': '2000-02-09T21:58:13+00:00', 'source': 'data\\medical_book.pdf', 'total_pages': 65, 'page': 1, 'page_label': '2'}, page_content='© World Health 

In [10]:
#Split create text chunks

In [11]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks


In [12]:
text_chunks = text_split(extracted_data)
print("Length of my chunk:",len(text_chunks))

Length of my chunk: 192


In [13]:
text_chunks

[Document(metadata={'producer': 'Acrobat PDFWriter 4.0 for Windows', 'creator': 'Microsoft Word - WHO_DIAGNOSI_11gW97.doc', 'creationdate': 'D:20000209211438', 'title': 'WHO_DIAGNOSI_11gW97', 'moddate': '2000-02-09T21:58:13+00:00', 'source': 'data\\medical_book.pdf', 'total_pages': 65, 'page': 0, 'page_label': '1'}, page_content='WHO/NCD/NCS/99.2\nOriginal: English\nDistr.: General\nDefinition, Diagnosis\nand Classification\nof Diabetes Mellitus\nand its Complications\nReport of a WHO Consultation\nPart 1: Diagnosis and Classification of\nDiabetes Mellitus\nWorld Health Organization\nDepartment of Noncommunicable Disease Surveillance\nGeneva'),
 Document(metadata={'producer': 'Acrobat PDFWriter 4.0 for Windows', 'creator': 'Microsoft Word - WHO_DIAGNOSI_11gW97.doc', 'creationdate': 'D:20000209211438', 'title': 'WHO_DIAGNOSI_11gW97', 'moddate': '2000-02-09T21:58:13+00:00', 'source': 'data\\medical_book.pdf', 'total_pages': 65, 'page': 1, 'page_label': '2'}, page_content='© World Health 

In [14]:
#download embeding model

In [15]:
def download_huggingface_embedding():
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings = download_huggingface_embedding()

C:\Users\USER\AppData\Local\Temp\ipykernel_11220\732167232.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [17]:
embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [18]:
query_result = embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [19]:
query_result

[-0.010300815105438232,
 0.18307922780513763,
 0.03081119991838932,
 0.0044528404250741005,
 -0.0273361224681139,
 -0.033562567085027695,
 0.03763148933649063,
 -0.031573336571455,
 -0.003391013015061617,
 -0.008950837887823582,
 0.03803616017103195,
 -0.05129104107618332,
 0.0003682934329845011,
 -0.023727091029286385,
 0.0927102193236351,
 -0.027795813977718353,
 -0.035152580589056015,
 -0.003224145621061325,
 -0.0768178328871727,
 -0.05761215463280678,
 0.07257603853940964,
 0.11128544807434082,
 0.01605846732854843,
 0.015908440575003624,
 -0.08232704550027847,
 0.00700728502124548,
 0.02901311218738556,
 0.0011386689729988575,
 0.1167173683643341,
 -0.032327331602573395,
 -0.032271552830934525,
 -0.0012590086553245783,
 0.10591629147529602,
 0.023600876331329346,
 0.009664947167038918,
 0.09834080189466476,
 0.042936310172080994,
 -0.019547604024410248,
 0.019267888739705086,
 -0.06417104601860046,
 0.023923343047499657,
 -0.05288001894950867,
 -0.026469500735402107,
 0.0055487076

In [21]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot2"  # ✅ all lowercase and hyphen

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # for all-MiniLM-L6-v2
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-west-2')
    )

index = pc.Index(index_name)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '1020a2b8211b4793d5e14927293ceabb', 'date': 'Wed, 02 Jul 2025 11:18:46 GMT', 'server': 'Google Frontend', 'Content-Length': '200', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Bad request: Your free plan does not support indexes in the us-west-2 region of aws. To create indexes in this region, upgrade your plan."},"status":400}
